In [2]:
%matplotlib inline
import numpy as np
from settings import Settings
from utils import *
from data_processor import data_processor
from augment_patient import augment_patient

lungPatients, labels = load_data()

size = Settings.size
no_slices = Settings.no_slices

imageData = []
for num, patient in enumerate(lungPatients):
    if num % 100 == 0:
        print('Saved -', num)
    try:
        patient_data = []
        patient_data.append(data_processor(patient=patient, labels_df=labels, size=size, noslices=no_slices, visualize=False))
        
        if get_label(patient, labels) == 6:
            print('Cancer')
            for i in range(Settings.aug_counter):
                print('Augmented image #{}'.format(i))
                patient_data.append(augment_patient(patient=patient, labels_df=labels, iteration=i, size=size, noslices=no_slices, visualize=False))

        for each in patient_data:
            print(each['img_data'].shape)
            imageData.append(each['img_data']) #, each['label'], each['patient']])
    except KeyError as e:
        print('Data is unlabeled')
    except Exception as er:
        print(er)

        
##Results are saved as numpy file
np.save('imageDataNew-{}-{}-{}-test.npy'.format(size, size, no_slices), imageData)

Saved - 0
(1, 64, 64)
(1, 64, 64)
(1, 64, 64)
Data is unlabeled


KeyboardInterrupt: 